In [ ]:
!pip install sqlalchemy==1.4.46
!pip install pandasql
!pip install geopy
!pip install -U kaleido
import pandas as pd
import pandasql as ps
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.4 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.28
    Uninstalling SQLAlchemy-2.0.28:
      Successfully uninstalled SQLAlchemy-2.0.28
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.46 which is incompatible.
  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26771 sha256=09e92b73846d1333d1f9ee8185d35c55fc3b08bff02c60797b4d02332bbb8ced
  Stored in directory: /root/.cache/pip/wheels/e9/bc/3a/8434bdcccf5779e72894a9b24fecbdcaf97940607eaf4bcdf9
Successfully built pandasql
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.3 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
crime_df = pd.read_csv("/content/drive/MyDrive/Crime_Data_from_2020_to_Present.csv")
crime_df

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,190326475,03/01/2020 12:00:00 AM,03/01/2020 12:00:00 AM,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,...,AA,Adult Arrest,510.0,998.0,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506
1,200106753,02/09/2020 12:00:00 AM,02/08/2020 12:00:00 AM,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330.0,998.0,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628
2,200320258,11/11/2020 12:00:00 AM,11/04/2020 12:00:00 AM,1700,3,Southwest,356,1,480,BIKE - STOLEN,...,IC,Invest Cont,480.0,NaN,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002
3,200907217,05/10/2023 12:00:00 AM,03/10/2020 12:00:00 AM,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,IC,Invest Cont,343.0,NaN,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387
4,220614831,08/18/2022 12:00:00 AM,08/17/2020 12:00:00 AM,1200,6,Hollywood,666,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,1900 TRANSIENT,NaN,34.0944,-118.3277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
910702,242106032,02/20/2024 12:00:00 AM,02/17/2024 12:00:00 AM,2330,21,Topanga,2103,1,420,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),...,IC,Invest Cont,420.0,NaN,NaN,NaN,22700 NAPA ST,NaN,34.2260,-118.6214
910703,241604246,01/08/2024 12:00:00 AM,01/08/2024 12:00:00 AM,800,16,Foothill,1601,2,625,OTHER ASSAULT,...,IC,Invest Cont,625.0,NaN,NaN,NaN,VAUGHN ST,DE GARMO AV,34.2865,-118.4161
910704,242004546,01/16/2024 12:00:00 AM,01/16/2024 12:00:00 AM,1510,20,Olympic,2013,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,HOBART BL,3RD ST,34.0690,-118.3054
910705,240705409,02/02/2024 12:00:00 AM,01/31/2024 12:00:00 AM,30,7,Wilshire,784,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,2300 S SYCAMORE AV,NaN,34.0360,-118.3527


In [ ]:
crime_df.describe(include='all')
crime_df.shape

(910707, 28)

In [ ]:
# seperate R2
R2_area = crime_df[['AREA','AREA NAME']].drop_duplicates()
R2_area_sorted = R2_area.sort_values(by='AREA').reset_index(drop=True)
R2_area_sorted.to_csv('/content/drive/MyDrive/550 CrimeLA data/R2_area.csv', index=False)

R1_main = crime_df.drop(['AREA NAME'], axis=1)

print(R2_area_sorted)
print(R2_area_sorted.shape) # 21 different areas

    AREA    AREA NAME
0      1      Central
1      2      Rampart
2      3    Southwest
3      4   Hollenbeck
4      5       Harbor
5      6    Hollywood
6      7     Wilshire
7      8      West LA
8      9     Van Nuys
9     10  West Valley
10    11    Northeast
11    12  77th Street
12    13       Newton
13    14      Pacific
14    15  N Hollywood
15    16     Foothill
16    17   Devonshire
17    18    Southeast
18    19      Mission
19    20      Olympic
20    21      Topanga
(21, 2)


In [ ]:
# seperate R3
R3_crime = crime_df[['Crm Cd','Crm Cd Desc']].drop_duplicates()
R3_crime_sorted = R3_crime.sort_values(by='Crm Cd').reset_index(drop=True)
R3_crime_sorted.to_csv('/content/drive/MyDrive/550 CrimeLA data/R3_crime.csv', index=False)

R1_main = R1_main.drop(['Crm Cd Desc'], axis=1)

print(R3_crime_sorted)
print(R3_crime_sorted.shape) # 139 different crimes

     Crm Cd                                        Crm Cd Desc
0       110                                  CRIMINAL HOMICIDE
1       113                            MANSLAUGHTER, NEGLIGENT
2       121                                     RAPE, FORCIBLE
3       122                                    RAPE, ATTEMPTED
4       210                                            ROBBERY
..      ...                                                ...
134     949                                    ILLEGAL DUMPING
135     950  DEFRAUDING INNKEEPER/THEFT OF SERVICES, OVER $...
136     951  DEFRAUDING INNKEEPER/THEFT OF SERVICES, $950 &...
137     954                                       CONTRIBUTING
138     956            LETTERS, LEWD  -  TELEPHONE CALLS, LEWD

[139 rows x 2 columns]
(139, 2)


In [ ]:
# seperate R4
R4_premis = crime_df[['Premis Cd','Premis Desc']].drop_duplicates().dropna()
R4_premis_sorted = R4_premis.sort_values(by='Premis Cd').reset_index(drop=True)
R4_premis_sorted.to_csv('/content/drive/MyDrive/550 CrimeLA data/R4_premis.csv', index=False)

R1_main = R1_main.drop(['Premis Cd'], axis=1)

print(R4_premis_sorted)
print(R4_premis_sorted.shape) # 307 different premis (places)

     Premis Cd                         Premis Desc
0        101.0                              STREET
1        102.0                            SIDEWALK
2        103.0                               ALLEY
3        104.0                            DRIVEWAY
4        105.0             PEDESTRIAN OVERCROSSING
..         ...                                 ...
302      967.0         MTA - GOLD LINE - CHINATOWN
303      968.0   MTA - GOLD LINE - LINCOLN/CYPRESS
304      969.0       MTA - GOLD LINE - HERITAGE SQ
305      970.0  MTA - GOLD LINE - SOUTHWEST MUSEUM
306      971.0     MTA - GOLD LINE - HIGHLAND PARK

[307 rows x 2 columns]
(307, 2)


In [ ]:
# seperate R5
R5_weapon = crime_df[['Weapon Used Cd','Weapon Desc']].drop_duplicates().dropna()
R5_weapon_sorted = R5_weapon.sort_values(by='Weapon Used Cd').reset_index(drop=True)
R5_weapon_sorted.to_csv('/content/drive/MyDrive/550 CrimeLA data/R5_weapon.csv', index=False)

R1_main = R1_main.drop(['Weapon Desc'], axis=1)

print(R5_weapon_sorted)
print(R5_weapon_sorted.shape) # 79 weapons

    Weapon Used Cd                 Weapon Desc
0            101.0                    REVOLVER
1            102.0                    HAND GUN
2            103.0                       RIFLE
3            104.0                     SHOTGUN
4            105.0     SAWED OFF RIFLE/SHOTGUN
..             ...                         ...
74           512.0           MACE/PEPPER SPRAY
75           513.0                    STUN GUN
76           514.0                   TIRE IRON
77           515.0           PHYSICAL PRESENCE
78           516.0  DOG/ANIMAL (SIC ANIMAL ON)

[79 rows x 2 columns]
(79, 2)


In [ ]:
# seperate R6
R6_status = crime_df[['Status','Status Desc']].drop_duplicates().reset_index(drop=True)
R6_status.to_csv('/content/drive/MyDrive/550 CrimeLA data/R6_status.csv', index=False)

R1_main = R1_main.drop(['Status Desc'], axis=1)

print(R6_status)
print(R6_status.shape) # 6 different status

  Status   Status Desc
0     AA  Adult Arrest
1     IC   Invest Cont
2     JA    Juv Arrest
3     AO   Adult Other
4     JO     Juv Other
5     CC           UNK
(6, 2)


In [ ]:
# check R1
print(R1_main.shape)

# drop exact location info and sub-area info to generalize to area level
R1_crime = R1_main.drop(columns=['LOCATION', 'Cross Street', 'LAT', 'LON'])
R1_crime = R1_crime.drop(columns=['Rpt Dist No', 'Part 1-2'])

(910707, 23)


In [ ]:
R1_crime = R1_crime.drop(columns=['Mocodes'])

In [ ]:
# Check data types of 'Date Rptd' and 'DATE OCC' columns
date_rptd_dtype = R1_crime['Date Rptd'].dtype
date_occ_dtype = R1_crime['DATE OCC'].dtype

# Display the data types
print(f"'Date Rptd' data type: {date_rptd_dtype}")
print(f"'DATE OCC' data type: {date_occ_dtype}")

'Date Rptd' data type: object
'DATE OCC' data type: object


In [ ]:
# Convert 'Date Rptd' and 'DATE OCC' to datetime
R1_crime['Date Rptd'] = pd.to_datetime(R1_crime['Date Rptd'], format='%m/%d/%Y %I:%M:%S %p')
R1_crime['DATE OCC'] = pd.to_datetime(R1_crime['DATE OCC'], format='%m/%d/%Y %I:%M:%S %p')

# Extract date components for 'Date Rptd' and 'DATE OCC'
R1_crime['Date Rptd'] = R1_crime['Date Rptd'].dt.date
R1_crime['Date Occ'] = R1_crime['DATE OCC'].dt.date

In [ ]:
# For the 'TIME OCC', assuming it's in HHMM format and needs to be converted to a 24-hour time string
# This assumes 'TIME OCC' is an integer. If it's stored differently, this conversion might need adjusting.
R1_crime['Time Occ'] = R1_crime['TIME OCC'].apply(lambda x: "{:02d}:{:02d}".format(x // 100, x % 100))

In [ ]:
R1_crime.drop(columns=['DATE OCC','TIME OCC'])

,DR_NO,Date Rptd,AREA,Crm Cd,Vict Age,Vict Sex,Vict Descent,Premis Desc,Weapon Used Cd,Status,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,Date Occ,Time Occ
0,190326475,2020-03-01,7,510,0,M,O,STREET,NaN,AA,510.0,998.0,NaN,NaN,2020-03-01,21:30
1,200106753,2020-02-09,1,330,47,M,O,BUS STOP/LAYOVER (ALSO QUERY 124),NaN,IC,330.0,998.0,NaN,NaN,2020-02-08,18:00
2,200320258,2020-11-11,3,480,19,X,X,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,IC,480.0,NaN,NaN,NaN,2020-11-04,17:00
3,200907217,2023-05-10,9,343,19,M,O,CLOTHING STORE,NaN,IC,343.0,NaN,NaN,NaN,2020-03-10,20:37
4,220614831,2022-08-18,6,354,28,M,H,SIDEWALK,NaN,IC,354.0,NaN,NaN,NaN,2020-08-17,12:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
910702,242106032,2024-02-20,21,420,44,F,O,DRIVEWAY,NaN,IC,420.0,NaN,NaN,NaN,2024-02-17,23:30
910703,241604246,2024-01-08,16,625,42,F,H,STREET,400.0,IC,625.0,NaN,NaN,NaN,2024-01-08,08:00
910704,242004546,2024-01-16,20,624,80,F,O,BUS STOP,400.0,IC,624.0,NaN,NaN,NaN,2024-01-16,15:10
910705,240705409,2024-02-02,7,510,0,NaN,NaN,STREET,NaN,IC,510.0,NaN,NaN,NaN,2024-01-31,00:30


In [ ]:
R1_crime.to_csv('/content/drive/MyDrive/550 CrimeLA data/R1_crime.csv', index=False)

In [ ]:
# Find rows where 'Crime Cd' is not equal to 'Crime Cd 1'
different_row = R1_crime.loc[R1_crime['Crm Cd'] != R1_crime['Crm Cd 1']]

# Display the first such row, if any
if not different_row.empty:
    print(different_row.iloc[0])
else:
    print("All rows have 'Crime Cd' equal to 'Crime Cd 1'.")


DR_NO                          221908151
Date Rptd                     2022-04-12
DATE OCC             2020-10-01 00:00:00
TIME OCC                               1
AREA                                  19
Crm Cd                               821
Vict Age                               8
Vict Sex                               F
Vict Descent                           H
Premis Desc       SINGLE FAMILY DWELLING
Weapon Used Cd                     400.0
Status                                IC
Crm Cd 1                           812.0
Crm Cd 2                           821.0
Crm Cd 3                             NaN
Crm Cd 4                             NaN
Date Occ                      2020-10-01
Time Occ                           00:01
Name: 11, dtype: object
